# Access Requests (TEMPLATE)
This script is a template for obtaining and exporting the access requests for a single-point AOI based on a set of defined constraints. 

The cells that require the user to input parameters prior to running are marked with `USER INPUT REQUIRED` in the header. Searching for `USER` in the notebook will also identify where the user must enter inputs.

* **Author:** Hayley Pippin
* **Last updated:** July 18, 2023
* **Required input(s):**
    * `credentials.json`: JSON containing the user's Capella Console credentials.
    * `JSON` or `GEOJSON` single point file
* **Output(s):**
    * CSV of access requests

## Setup
Unless otherwise noted, these cells must be run every time you use this script. 

### Install packages
The following cell **only needs to be run once** if packages are not already installed. Uncomment any of the following lines to install the necessary packages.

In [ ]:
#!pip install requests
#!pip install json
#!pip install pandas

### Import packages and define helper functions

In [ ]:
import requests
import json
import csv
import os
import datetime
import pandas as pd

# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))

### Authentication (USER INPUT REQUIRED)
This cell needs to be run hourly to re-authenticate with the Capella system.

In [ ]:
# Load username and password
with open('WRITE PATH HERE') as f: # USER: Input path to credentials.json file.
    data = json.load(f)
    username = data['username']
    password = data['password']

# Get a valid token from the auth service
r = requests.post("https://api.capellaspace.com/token", 
                  headers = {'Content-Type': 'application/x-www-form-urlencoded'}, auth=(username,password))
access_token = r.json()["accessToken"]
# p(accessToken)

# GET user ID and org ID
headers = {'Authorization':'Bearer ' + access_token}
r = requests.get("https://api.capellaspace.com/user", headers=headers)
user_id = r.json()["id"]
org_id = r.json()["organizationId"]
#p(r.json())

# Print user and org ID
print('User email: ', r.json()['email'], '\nOrganization: ', r.json()['organization']['name'], '\nEnvironment: ', r.json()['apiEnvironmentRole'])

## Determine Access Requests

### Define access request input parameters (USER INPUT REQUIRED)

In [ ]:
# Set AOI (must be SINGLE POINT in JSON or GEOJSON format)
aoi_path = "WRITE PATH HERE" # USER: Input path to AOI.
aoi = json.load(open(aoi_path))
aoi_geom = aoi['features'][0]['geometry'] # Get the geometry of the AOI

# Name and description
request_name = '' # USER: Set the name of the request.
request_description = '' # USER: Set a description for the request.

# Access request window
# Default: 30-day window starting at current UTC date/time
start_date = datetime.datetime.utcnow()
end_date = start_date + datetime.timedelta(days = 30)
# USER: To change the window from the default, uncomment and set the start_date and end_date for your desired window in UTC in YYYY,M,D format.
# start_date = datetime.date(YYYY,M,D)
# end_date = datetime.date(YYYY,M,D)
window_open = start_date.strftime("%Y-%m-%dT%H:%M:%SZ") # format start date
window_close = end_date.strftime("%Y-%m-%dT%H:%M:%SZ") # format end date

# USER: Set constraints below.
orbital_plane = [45, 53, 97] # options: 45, 53, 97
orbit_state = "either" # options: ascending, descending, either
look_direction = "either" # options: left, right, either
look_angle_min = 15 # full: 5, extended: 15, standard: 25
look_angle_max = 50 # max: 50
img_width = 8000 # in meters
img_length = 100000 # in meters

### Create and `POST` access request

In [ ]:
# Define access request
access_request = {
    "geometry": aoi_geom,
    "properties": {
      "userId": user_id,
      "orgId": org_id,
      "accessrequestDescription": request_description,
      "accessrequestName": request_name,
      "windowOpen": window_open,
      "windowClose": window_close,
      "accessConstraints": {
          "ascDsc": orbit_state,
          "lookDirection": look_direction,
          "orbitalPlanes": orbital_plane,
          "offNadirMin": look_angle_min,
          "offNadirMax": look_angle_max,
          "imageWidth": img_width,
          "imageLength": img_length
          # "localTime": [
          #   [
          #     0,
          #     21600
          #   ],
          #   [
          #     64800,
          #     86400
          #   ]
          # ]
      }
    },
    "type": "Feature"
}

# POST request
headers = {'Authorization':'Bearer ' + access_token}
r = requests.post("https://api.capellaspace.com/ma/accessrequests/", json = access_request, headers = headers)
p(r.json())

### Print access request info and `GET` current status
Re-run the cell below until `ar_status` = `completed` and `ar_accessibility` = `accessible` to move on to next section (can take a couple minutes)

In [ ]:
# Extract access request properties of interest 
ar_id = r.json()["properties"]["accessrequestId"]
ar_constraints = r.json()["properties"]["accessConstraints"]
ar_status = r.json()["properties"]["processingStatus"]
ar_accessibility = r.json()["properties"]["accessibilityStatus"]

# Print request ID
print("Request ID: ", ar_id)

# GET access request status
r = requests.get("https://api.capellaspace.com/ma/accessrequests/" + ar_id, headers = headers)
print("Processing status: ", ar_status, "\nAccessibility status: ", ar_accessibility, "\nConstraints: ")
p(ar_constraints)

## View and Export Access Requests

### Full Access Request Table

#### Get and view table of access requests

In [ ]:
# GET access requests
r = requests.get("https://api.capellaspace.com/ma/accesses/" + ar_id, headers=headers)

# View requests as table
access_df = pd.json_normalize(r.json())
col_interest = ['spacecraftId', 'windowOpen', 'windowClose', 'orbitalPlane', 'accessProperties.ascdsc', 
                'accessProperties.lookDirection', 'accessProperties.offNadirMin']

#access_df # Uncomment to view full table of access requests
access_df[col_interest] # Uncomment to view access requests only with columns of interest

#### Export access requests as CSV (USER INPUT REQUIRED)

In [ ]:
# Inputs
csv_path = "WRITE PATH HERE" # USER: Input path to directory to store CSV of access requests. 
csv_filename = "WRITE NAME HERE" # USER: Input name for CSV. Example: 'aoi-access-requests.csv'

# Clean access request dataframe for export
# Set column names
access_df.columns = ['Center WGS84', 'Center ECEF', 'Spacecraft ID', 'Access ID', 'Access Request ID', 'Tile ID',
                     'Window Open (UTC)', 'Window Close (UTC)', 'Window Open (Local)', 'Window Close (Local)', 
                     'Orbital Plane', 'Orbit State', 'Look Direction', 'Local Mean Time', 'Az Open', 'Az Close', 
                     'Elev Min', 'Elev Max', 'Off-Nadir Min', 'Off-Nadir Max']
# Filter dataframe to columns, rows of interest
access_df_filt = access_df.drop('Tile ID', axis = 1)

# Create and export CSV
access_df_filt.to_csv(csv_path + csv_filename)

### Summary Access Request Table

#### Generate summary table of access requests
Shows the count of all possible image configurations during the specified time period.

In [ ]:
access_df_summary = access_df.groupby(['orbitalPlane',
            'accessProperties.ascdsc', 
            'accessProperties.lookDirection'
           ]).agg(passCount=('windowOpen', 'count')
                 #  LookDirAzimuth=('LookDirectionAzimuth[degrees]', 'mean'),
                 # LookDirAzimuthStd=('LookDirectionAzimuth[degrees]', 'std'),
                 #  SatAzimuth = ('SatelliteAzimuth[degrees]', 'mean')
                 ).reset_index().rename(columns = {'accessProperties.ascdsc': 'orbitState', 'accessProperties.lookDirection': 'lookDirection'})
access_df_summary

#### Export summary table as CSV (USER INPUT REQUIRED)

In [ ]:
# Inputs
csv_sum_path = "WRITE PATH HERE" # USER: Input path to directory to store summary CSV of access requests. 
csv_sum_filename = "WRITE NAME HERE" # USER: Input name for summary CSV. Example: 'aoi-access-requests-summary.csv'

# Export
access_df_summary.to_csv(csv_sum_path + csv_sum_filename)